# GPT-5 新しいパラメータとツール

GPT-5シリーズでは、モデルの応答をより細かく制御できる新しい開発者向けコントロールを導入しています。出力の長さやスタイルの調整から、厳密なフォーマットの強制まで、幅広い制御が可能です。以下は最新機能の概要です：

| # | 機能 | 概要 | 値 / 使用方法 |
|----|---------|----------|----------------|
| 1. | **冗長性パラメータ** | モデルの回答をより詳細にしたり、簡潔にしたりするヒントを与えます。プロンプトを安定させ、書き直しの代わりにパラメータを使用できます。 | • **low** → 簡潔なUX、最小限の文章<br>• **medium** *(デフォルト)* → バランスの取れた詳細度<br>• **high** → 冗長、監査、教育、引き継ぎに最適 |
| 2. | **自由形式関数呼び出し** | PythonスクリプトからSQLクエリまで、生のテキストペイロードをJSONラッピングなしで直接カスタムツールに生成します。以下のような外部ランタイムに対してより大きな柔軟性を提供：<br>• コードサンドボックス（Python、C++、Java等）<br>• SQLデータベース<br>• シェル環境<br>• 設定ジェネレータ | 構造化されたJSONが不要で、生のテキストがターゲットツールにとってより自然な場合に使用します。 |
| 3. | **文脈自由文法（CFG）** | 言語における有効な文字列を定義する生成規則のセット。各規則は、周囲の文脈に依存せずに、非終端記号を終端記号や他の非終端記号に書き換えます。プログラミング言語やOpenAIツールのカスタム形式の構文に出力を制約するのに有用です。 | 文法で受け入れられる有効な文字列のみをモデルが出力することを保証する契約として使用します。 |
| 4. | **最小推論** | レイテンシを最小化し、最初のトークンまでの時間を短縮するため、推論トークンを少なくまたは全く使わずにGPT-5を実行します。説明が不要な決定論的で軽量なタスク（抽出、フォーマット、短い書き換え、単純な分類）に最適です。指定されない場合、努力レベルはmediumがデフォルトです。 | 推論努力を「minimal」に設定。マルチステップ計画やツール集約的なワークフローでは避けてください。 |

**対応モデル：**
- gpt-5
- gpt-5-mini
- gpt-5-nano

**対応APIエンドポイント**
- Responses API
- Chat Completions API

注意：GPT-5シリーズのモデルから最高のパフォーマンスを得るために、Responses APIの使用を推奨します。

## 前提条件

GPT-5の新しいパラメータとツールをサポートするOpenAI SDKの更新から始めましょう。環境変数として`OPENAI_API_KEY`を設定していることを確認してください。

In [2]:
!pip install --quiet --upgrade openai pandas && \
echo -n "openai " && pip show openai | grep '^Version:' | cut -d' ' -f2 && \
echo -n "pandas " && pip show pandas | grep '^Version:' | cut -d' ' -f2

openai 1.99.2
pandas 2.3.1


## 1. Verbosity パラメータ

### 1.1 概要
verbosity パラメータを使用すると、モデルの回答をより詳細にしたり、より簡潔にしたりするヒントを与えることができます。

**値:** "low", "medium", "high"

- low → 簡潔なUX、最小限の文章。
- medium（デフォルト）→ バランスの取れた詳細レベル。
- high → 冗長、監査、教育、引き継ぎに最適。

プロンプトを安定させ、書き直しではなくパラメータを使用してください。

In [3]:
from openai import OpenAI
import pandas as pd
from IPython.display import display

client = OpenAI()

question = "Write a poem about a boy and his first pet dog."

data = []

for verbosity in ["low", "medium", "high"]:
    response = client.responses.create(
        model="gpt-5-mini",
        input=question,
        text={"verbosity": verbosity}
    )

    # Extract text
    output_text = ""
    for item in response.output:
        if hasattr(item, "content"):
            for content in item.content:
                if hasattr(content, "text"):
                    output_text += content.text

    usage = response.usage
    data.append({
        "Verbosity": verbosity,
        "Sample Output": output_text,
        "Output Tokens": usage.output_tokens
    })

# Create DataFrame
df = pd.DataFrame(data)

# Display nicely with centered headers
pd.set_option('display.max_colwidth', None)
styled_df = df.style.set_table_styles(
    [
        {'selector': 'th', 'props': [('text-align', 'center')]},  # Center column headers
        {'selector': 'td', 'props': [('text-align', 'left')]}     # Left-align table cells
    ]
)

display(styled_df)


,Verbosity,Sample Output,Output Tokens
0,low,"He found a scruff of fur behind the shed one spring afternoon, a heartbeat small and fast beneath a coat of dust and light. The world shrank to two—mud on sneakers, a wag, a clumsy tune— names rolled off his tongue like marbles, simple, sure, and bright. They learned the map of each other's hands: the scratch beneath the ear, the way a storm could change the shape of brave into a shake. Mornings were for toast and sunlight, afternoons for running near the riverbank where leaves applauded every leap they'd take. At night they shared a blanket and the secret of the dark, the boy with whispered stories, the dog with steady breath. Years braided into footprints—first skinned knees, then a spark of barnyard gray upon a muzzle, slow and gentle as a wreath. When time unlatched its gates, the boy still carried small things: a collar, a chewed shoe, the echo of a bark that taught him how to hope. He learned that love can look like leaving crumbs of ordinary kings, and that some firsts fit in your pockets long after they have gone.",560
1,medium,"He found him folded in the crook of a cardboard box, a tiny ribcage hitching like a thought. The boy had pockets full of pennies and promises; the dog had eyes like two small questions. They learned names together — the boy said one, the dog tilted his head and accepted it. Mornings were clumsy lessons: leash in hand, the dog discovering sidewalks with a sneeze of wonder, the boy discovering courage at the end of a rope. They chased afternoons into puddles, mud kissing the boy's knees and the dog's whiskers. The dog taught him how to throw sticks that never came back and how to forgive them when they didn't. Evenings were for quiet conspiracies: the dog's breath a warm punctuation against the boy's ankle as the sky grew blue-black. Homework became a small island between their worlds, a pencil, a pat, the faithful presence of paws on carpet. The dog learned how to sit for apples, how to hide a cold nose under a blanket of fingers. The boy learned how to stitch up a torn stuffed bear, how to say sorry and mean it. There were days of thunder when the boy's knees knocked, and the dog, all stern responsibility, pressed his head into the hollow of the boy's fear and held it there as if he could anchor lightning with his chin. They practiced being brave together: doors opened for new schools, new roads, a first bike without training wheels, the dog a steady metronome of tail and warmth, never asking to be anything but present. Seasons unraveled the way they always do. Snow came to lay white questions across the yard; summer stretched its lazy hands and left grass bleaching in August. The boy grew taller and later, the dog moved slower, but in late afternoons they still shared the same light — a private currency of sun and shadow. When the boy learned the language of goodbyes, it was the dog who taught him how to soften them. A last look, a lingering hand across the coat, and a promise that out of all the small ordinary days something invincible had been braided: two hearts, a leash, a map of pawprints on the threshold. Years later, the boy — now grown — tucks a photograph into his coat pocket. He feels the hollow where a warm head used to rest and smiles. Some bonds refuse to be folded away. In the quiet hum of rememberings, he can still hear a collar's jingle and a small, glad bark: first home, first friend, first forever.",849
2,high,"The day the boy met his dog the world grew wider— a small breath of fur and a damp, earnest nose pressed like a secret against his palm. They stood on the porch and the sun tilted curious, as if the sky had come to see how two new things might fit together. He named him after a comic-strip hero, or maybe he didn’t name him at all at first, just laughed and let the sound of it become a name. They learned each other’s weight: the dog’s heavy joy, the boy’s thin, cautious fingers turning into hands that could hold a leaping he

出力トークンは冗長性にほぼ線形にスケールします：低（560）→ 中（849）→ 高（1288）。

### 2.3 コーディング用途での詳細度の使用

詳細度パラメータは、生成されるコードの長さと複雑さ、および付随する説明の深さにも影響を与えます。以下は、100万個のランダムな数値の配列をソートするPythonプログラムを生成するタスクで、様々な詳細度レベルを使用した例です。

In [4]:
from openai import OpenAI

client = OpenAI()

prompt = "Output a Python program that sorts an array of 1000000 random numbers"

def ask_with_verbosity(verbosity: str, question: str):
    response = client.responses.create(
        model="gpt-5-mini",
        input=question,
        text={
            "verbosity": verbosity
        }
    )

    # Extract assistant's text output
    output_text = ""
    for item in response.output:
        if hasattr(item, "content"):
            for content in item.content:
                if hasattr(content, "text"):
                    output_text += content.text

    # Token usage details
    usage = response.usage

    print("--------------------------------")
    print(f"Verbosity: {verbosity}")
    print("Output:")
    print(output_text)
    print("Tokens => input: {} | output: {}".format(
        usage.input_tokens, usage.output_tokens
    ))


# Example usage:
ask_with_verbosity("low", prompt)

--------------------------------
Verbosity: low
Output:
```python
#!/usr/bin/env python3
import random
import time

def main():
    N = 1_000_000
    arr = [random.random() for _ in range(N)]

    t0 = time.perf_counter()
    arr.sort()
    t1 = time.perf_counter()

    print(f"Sorted {N} numbers in {t1 - t0:.4f} seconds")
    print("First 10:", arr[:10])
    print("Last 10:", arr[-10:])

if __name__ == "__main__":
    main()
```
Tokens => input: 21 | output: 575


コードの出力が単純なスクリプトであることに注目してください。では、'medium'で実行してみましょう。

In [5]:
ask_with_verbosity("medium", prompt)

--------------------------------
Verbosity: medium
Output:
Here's a simple Python script that generates 1,000,000 random numbers, sorts them using the built-in Timsort, and reports timings and a small sample of the sorted output:

```python
#!/usr/bin/env python3
import random
import time

def main():
    N = 1_000_000
    random.seed(42)  # remove or change for different runs

    t0 = time.perf_counter()
    data = [random.random() for _ in range(N)]
    t1 = time.perf_counter()

    data.sort()
    t2 = time.perf_counter()

    # Basic verification and sample output
    is_sorted = all(data[i] <= data[i+1] for i in range(len(data)-1))
    print(f"Generated {N} random numbers in {t1 - t0:.3f} seconds")
    print(f"Sorted in {t2 - t1:.3f} seconds")
    print("Sorted check:", is_sorted)
    print("First 10 values:", data[:10])
    print("Last 10 values:", data[-10:])

if __name__ == "__main__":
    main()
```

Notes:
- This uses Python's built-in list sort (Timsort), which is efficient

中程度の詳細度では、追加の説明を含むより豊富なコードが生成されました。高い詳細度でも同じことを試してみましょう。

In [6]:
ask_with_verbosity("high", prompt)

--------------------------------
Verbosity: high
Output:
Here's a single, self-contained Python program that generates 1,000,000 random numbers and sorts them. It supports two backends: the built-in Python list sort (Timsort) and NumPy (if you have NumPy installed). It measures and prints the time taken for generation, sorting, and verification.

Copy the code into a file (for example sort_random.py) and run it. By default it uses the pure Python backend; pass --backend numpy to use NumPy.

Note: Sorting a million Python floats uses a moderate amount of memory (Python floats and list overhead). NumPy will typically be faster and use less overhead but requires the numpy package.

Program:

import time
import random
import argparse
import sys

def is_sorted_list(a):
    # Linear check for sortedness
    return all(a[i] <= a[i+1] for i in range(len(a)-1))

def main():
    parser = argparse.ArgumentParser(description="Generate and sort random numbers.")
    parser.add_argument("--n", type=

高い詳細度では、追加の詳細と説明が提供されました。

### 1.3 要点

新しい詳細度パラメータは、**基盤となるプロンプトを変更することなく**、正確性と推論品質を保ちながら、モデルの出力の長さと深さの両方を確実にスケールします。
この例では：

- **低い詳細度**では、余分なコメントや構造のない最小限の機能的なスクリプトが生成されます。
- **中程度の詳細度**では、説明的なコメント、関数構造、再現性制御が追加されます。
- **高い詳細度**では、引数解析、複数のソート方法、タイミング/検証、使用方法の注記、ベストプラクティスのヒントを含む包括的で本番環境対応のスクリプトが生成されます。

## 2. フリーフォーム関数呼び出し

### 2.1 概要
GPT-5は、新しいツール`"type": "custom"`を使用して、PythonスクリプトからSQLクエリまで、あらゆる生のテキストペイロードをJSONでデータをラップすることなく、カスタムツールに送信できるようになりました。これは従来の構造化された関数呼び出しとは異なり、以下のような外部ランタイムとやり取りする際により大きな柔軟性を提供します：

- サンドボックス付きのcode_exec（Python、C++、Java、...）
- SQLデータベース
- シェル環境
- 設定ジェネレーター

**カスタムツールタイプは並列ツール呼び出しをサポートしていないことに注意してください。**

### 2.2 クイックスタート例 - 円の面積を計算する

以下のコードは、円の面積を計算するシンプルなPythonコードを生成し、モデルに自由形式のツール呼び出しを使用して結果を出力するよう指示します。

In [7]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    input="Please use the code_exec tool to calculate the area of a circle with radius equal to the number of 'r's in strawberry",
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "code_exec",
            "description": "Executes arbitrary python code",
        }
    ]
)
print(response.output)

[ResponseReasoningItem(id='rs_6894e31b1f8081999d18325e5aeffcfe0861a2e1728d1664', summary=[], type='reasoning', content=[], encrypted_content=None, status=None), ResponseCustomToolCall(call_id='call_Gnqod2MwPvayp2JdNyA0z0Ah', input='# Counting \'r\'s in the word "strawberry" and computing circle area with that radius\nimport math\nr = "strawberry".count(\'r\')\narea = math.pi * r**2\n{"radius": r, "area": area, "area_exact": f"{r}*pi"}', name='code_exec', type='custom_tool_call', id='ctc_6894e31c66f08199abd622bb5ac3c4260861a2e1728d1664', status='completed')]


モデルは生のPythonを含む`tool call`を出力します。そのコードをサーバーサイドで実行し、印刷された結果をキャプチャして、フォローアップのresponses.create呼び出しで送り返します。

### 2.3 ミニベンチマーク – 3つの言語での配列ソート
自由形式のツール呼び出しの使用を説明するために、GPT-5に以下のことを依頼します：
- 固定配列を10回ソートするPython、C++、Javaのコードを生成する
- コード内で各反復にかかった時間（ミリ秒）のみを出力する
- 3つの関数をすべて呼び出してから停止する

In [8]:
from openai import OpenAI
from typing import List, Optional

MODEL_NAME = "gpt-5"

# Tools that will be passed to every model invocation. They are defined once so
# that the configuration lives in a single place.
TOOLS = [
    {
        "type": "custom",
        "name": "code_exec_python",
        "description": "Executes python code",
    },
    {
        "type": "custom",
        "name": "code_exec_cpp",
        "description": "Executes c++ code",
    },
    {
        "type": "custom",
        "name": "code_exec_java",
        "description": "Executes java code",
    },
]

client = OpenAI()

def create_response(
    input_messages: List[dict],
    previous_response_id: Optional[str] = None,
):
    """Wrapper around ``client.responses.create``.

    Parameters
    ----------
    input_messages: List[dict]
        The running conversation history to feed to the model.
    previous_response_id: str | None
        Pass the ``response.id`` from the *previous* call so the model can keep
        the thread of the conversation.  Omit on the very first request.
    """
    kwargs = {
        "model": MODEL_NAME,
        "input": input_messages,
        "text": {"format": {"type": "text"}},
        "tools": TOOLS,
    }
    if previous_response_id:
        kwargs["previous_response_id"] = previous_response_id

    return client.responses.create(**kwargs)

# Recursive 
def run_conversation(
    input_messages: List[dict],
    previous_response_id: Optional[str] = None,
):
  
    response = create_response(input_messages, previous_response_id)

    # ``response.output`` is expected to be a list where element 0 is the model
    # message.  Element 1 (if present) denotes a tool call.  When the model is
    # done with tool calls, that element is omitted.
    tool_call = response.output[1] if len(response.output) > 1 else None

    if tool_call and tool_call.type == "custom_tool_call":
        print("--- tool name ---")
        print(tool_call.name)
        print("--- tool call argument (generated code) ---")
        print(tool_call.input)
        
        # Add a synthetic *tool result* so the model can continue the thread.
        
        input_messages.append(
            {
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": "done", # <-- replace with the result of the tool call
            }
        )

        # Recurse with updated conversation and track the response id so the
        # model is aware of the prior turn.
        return run_conversation(input_messages, previous_response_id=response.id)
    else:
        # Base-case: no further tool call - return. 
        return 


prompt = """
Write code to sort the array of numbers in three languages: C++, Python and Java (10 times each)using code_exec functions.

ALWAYS CALL THESE THREE FUNCTIONS EXACTLY ONCE: code_exec_python, code_exec_cpp and code_exec_java tools to sort the array in each language. Stop once you've called these three functions in each language once.

Print only the time it takes to sort the array in milliseconds. 

[448, 986, 255, 884, 632, 623, 246, 439, 936, 925, 644, 159, 777, 986, 706, 723, 534, 862, 195, 686, 846, 880, 970, 276, 613, 736, 329, 622, 870, 284, 945, 708, 267, 327, 678, 807, 687, 890, 907, 645, 364, 333, 385, 262, 730, 603, 945, 358, 923, 930, 761, 504, 870, 561, 517, 928, 994, 949, 233, 137, 670, 555, 149, 870, 997, 809, 180, 498, 914, 508, 411, 378, 394, 368, 766, 486, 757, 319, 338, 159, 585, 934, 654, 194, 542, 188, 934, 163, 889, 736, 792, 737, 667, 772, 198, 971, 459, 402, 989, 949]
"""

# Initial developer message.
messages = [
    {
        "role": "developer",
        "content": prompt,
    }
]

run_conversation(messages)


--- tool name ---
code_exec_python
--- tool call argument (generated code) ---
import time

arr = [448, 986, 255, 884, 632, 623, 246, 439, 936, 925, 644, 159, 777, 986, 706, 723, 534, 862, 195, 686, 846, 880, 970, 276, 613, 736, 329, 622, 870, 284, 945, 708, 267, 327, 678, 807, 687, 890, 907, 645, 364, 333, 385, 262, 730, 603, 945, 358, 923, 930, 761, 504, 870, 561, 517, 928, 994, 949, 233, 137, 670, 555, 149, 870, 997, 809, 180, 498, 914, 508, 411, 378, 394, 368, 766, 486, 757, 319, 338, 159, 585, 934, 654, 194, 542, 188, 934, 163, 889, 736, 792, 737, 667, 772, 198, 971, 459, 402, 989, 949]

start = time.perf_counter()
for _ in range(10):
    b = arr[:]  # copy
    b.sort()
elapsed_ms = int((time.perf_counter() - start) * 1000)
print(elapsed_ms, end="")
--- tool name ---
code_exec_cpp
--- tool call argument (generated code) ---
#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
using namespace std;

int main() {
    vector<int> a = {448, 986, 255, 884, 632, 6

モデルは同じアルゴリズムに対してPython、C++、Javaの3つのコードブロックを出力しました。関数呼び出しの出力は、すべての関数が正確に1回ずつ呼び出されるまでモデルが処理を続行できるよう、入力としてモデルに連鎖的にフィードバックされました。

### 2.4 まとめ

GPT-5のフリーフォームツール呼び出しでは、Pythonスクリプト、SQLクエリ、設定ファイルなどの生のテキストペイロードを、JSONラッピングなしで直接カスタムツールに送信できます。これにより、外部ランタイムとの相互作用においてより大きな柔軟性が提供され、モデルがツールが期待する正確な形式でコードやテキストを生成できるようになります。構造化されたJSONが不要で、自然なテキスト出力が使いやすさを向上させる場合に最適です。

## 3. 文脈自由文法（CFG）

### 3.1 概要
文脈自由文法は、どの文字列が言語に属するかを定義する生成規則の集合です。各規則は、非終端記号を終端記号（リテラルトークン）および/または他の非終端記号の列に書き換えます。これは周囲の文脈に依存しないため、文脈自由と呼ばれます。CFGはほとんどのプログラミング言語の構文を捉えることができ、OpenAIカスタムツールでは、モデルが文法で受け入れられる文字列のみを出力するよう強制する契約として機能します。

### 3.2 文法の基礎

**サポートされている文法構文**
- Lark - https://lark-parser.readthedocs.io/en/stable/
- Regex - https://docs.rs/regex/latest/regex/#syntax

内部的にLLGuidanceを使用してモデルのサンプリングを制約しています：https://github.com/guidance-ai/llguidance。

**サポートされていないLark機能**
- 正規表現での先読み・後読み（`(?=...)`、`(?!...)`など）
- 正規表現での遅延修飾子（`*?`、`+?`、`??`）
- 終端記号の優先度、テンプレート、%declares、%import（%import commonを除く）

**終端記号 vs 規則 & 貪欲字句解析**

| 概念 | ポイント |
|------|----------|
| 終端記号（大文字）| 字句解析器によって最初にマッチされる – 最長マッチが勝利 |
| 規則（小文字）| 終端記号を組み合わせる；テキストのトークン化方法に影響を与えることはできない |
| 貪欲字句解析器 | 複数の終端記号にまたがって自由テキストを「形成」しようとしてはいけない – 制御を失う |

**正しいパターン設計 vs 間違ったパターン設計**

✅ **1つの境界付き終端記号がアンカー間の自由テキストを処理**
```
start: SENTENCE
SENTENCE: /[A-Za-z, ]*(the hero|a dragon)[A-Za-z, ]*(fought|saved)[A-Za-z, ]*(a treasure|the kingdom)[A-Za-z, ]*\./
```
❌ **自由テキストを複数の終端記号/規則に分割しない**
```
start: sentence
sentence: /[A-Za-z, ]+/ subject /[A-Za-z, ]+/ verb /[A-Za-z, ]+/ object /[A-Za-z, ]+/
```

### 3.3 例 - SQL方言 — MS SQL vs PostgreSQL

以下のコード例は、CFGを使用してマルチ方言SQLツールを構築するための標準的なリファレンスです。これは以下を実演しています：

- TOPとLIMITのセマンティクスをエンコードする2つの独立した文法定義（`mssql_grammar_definition`、`postgres_grammar_definition`）
- 単一のスクリプトでプロンプト、呼び出し、ツール呼び出しの検査を行う方法
- アシスタントの応答の並列検査

異なるSQLダイアレクト用のLARK文法を定義する

In [9]:
import textwrap

# ----------------- grammars for MS SQL dialect -----------------
mssql_grammar = textwrap.dedent(r"""
            // ---------- Punctuation & operators ----------
            SP: " "
            COMMA: ","
            GT: ">"
            EQ: "="
            SEMI: ";"

            // ---------- Start ----------
            start: "SELECT" SP "TOP" SP NUMBER SP select_list SP "FROM" SP table SP "WHERE" SP amount_filter SP "AND" SP date_filter SP "ORDER" SP "BY" SP sort_cols SEMI

            // ---------- Projections ----------
            select_list: column (COMMA SP column)*
            column: IDENTIFIER

            // ---------- Tables ----------
            table: IDENTIFIER

            // ---------- Filters ----------
            amount_filter: "total_amount" SP GT SP NUMBER
            date_filter: "order_date" SP GT SP DATE

            // ---------- Sorting ----------
            sort_cols: "order_date" SP "DESC"

            // ---------- Terminals ----------
            IDENTIFIER: /[A-Za-z_][A-Za-z0-9_]*/
            NUMBER: /[0-9]+/
            DATE: /'[0-9]{4}-[0-9]{2}-[0-9]{2}'/
    """)

# ----------------- grammars for PostgreSQL dialect -----------------
postgres_grammar = textwrap.dedent(r"""
            // ---------- Punctuation & operators ----------
            SP: " "
            COMMA: ","
            GT: ">"
            EQ: "="
            SEMI: ";"

            // ---------- Start ----------
            start: "SELECT" SP select_list SP "FROM" SP table SP "WHERE" SP amount_filter SP "AND" SP date_filter SP "ORDER" SP "BY" SP sort_cols SP "LIMIT" SP NUMBER SEMI

            // ---------- Projections ----------
            select_list: column (COMMA SP column)*
            column: IDENTIFIER

            // ---------- Tables ----------
            table: IDENTIFIER

            // ---------- Filters ----------
            amount_filter: "total_amount" SP GT SP NUMBER
            date_filter: "order_date" SP GT SP DATE

            // ---------- Sorting ----------
            sort_cols: "order_date" SP "DESC"

            // ---------- Terminals ----------
            IDENTIFIER: /[A-Za-z_][A-Za-z0-9_]*/
            NUMBER: /[0-9]+/
            DATE: /'[0-9]{4}-[0-9]{2}-[0-9]{2}'/
    """)

### 3.4 特定のSQLダイアレクトの生成
プロンプトを定義し、関数を呼び出してMS SQLダイアレクトを生成しましょう

In [10]:
from openai import OpenAI
client = OpenAI()

sql_prompt_mssql = (
    "Call the mssql_grammar to generate a query for Microsoft SQL Server that retrieve the "
    "five most recent orders per customer, showing customer_id, order_id, order_date, and total_amount, "
    "where total_amount > 500 and order_date is after '2025-01-01'. "
)

response_mssql = client.responses.create(
    model="gpt-5",
    input=sql_prompt_mssql,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "mssql_grammar",
            "description": "Executes read-only Microsoft SQL Server queries limited to SELECT statements with TOP and basic WHERE/ORDER BY. YOU MUST REASON HEAVILY ABOUT THE QUERY AND MAKE SURE IT OBEYS THE GRAMMAR.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": mssql_grammar
            }
        },
    ],
    parallel_tool_calls=False
)

print("--- MS SQL Query ---")
print(response_mssql.output[1].input)

--- MS SQL Query ---
SELECT TOP 5 customer_id, order_id, order_date, total_amount FROM orders WHERE total_amount > 500 AND order_date > '2025-01-01' ORDER BY order_date DESC;


出力されたSQLは「SELECT TOP」構文を正確に使用しています。

In [11]:
sql_prompt_pg = (
    "Call the postgres_grammar to generate a query for PostgreSQL that retrieve the "
    "five most recent orders per customer, showing customer_id, order_id, order_date, and total_amount, "
    "where total_amount > 500 and order_date is after '2025-01-01'. "
)

response_pg = client.responses.create(
    model="gpt-5",
    input=sql_prompt_pg,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "postgres_grammar",
            "description": "Executes read-only PostgreSQL queries limited to SELECT statements with LIMIT and basic WHERE/ORDER BY. YOU MUST REASON HEAVILY ABOUT THE QUERY AND MAKE SURE IT OBEYS THE GRAMMAR.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": postgres_grammar
            }
        },
    ],
    parallel_tool_calls=False,
)

print("--- PG SQL Query ---")
print(response_pg.output[1].input)

--- PG SQL Query ---
SELECT customer_id, order_id, order_date, total_amount FROM orders WHERE total_amount > 500 AND order_date > '2025-01-01' ORDER BY order_date DESC LIMIT 5;


出力は同じ論理クエリを異なる物理構文で強調表示しています。選択した方言に対して有効な文のみをモデルが生成できるよう、明確な文法を提供してください。

| 方言          | 生成されたクエリ                                              | 主な違い                                |
|---------------|--------------------------------------------------------------|------------------------------------------|
| MS SQL Server | SELECT TOP 5 customer_id, … ORDER BY order_date DESC;         | 列リストの前に`TOP N`句を使用。          |
| PostgreSQL    | SELECT customer_id, … ORDER BY order_date DESC LIMIT 5;       | `ORDER BY`の後に`LIMIT N`を使用。        |

### 3.5 例 - Regex CFG構文

以下のコード例では、Regex CFG構文を使用して、フリーフォームツール呼び出しを特定のタイムスタンプパターンに制約する方法を示しています。

In [12]:
from openai import OpenAI
client = OpenAI()

timestamp_grammar_definition = r"^\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]) (?:[01]\d|2[0-3]):[0-5]\d$"

timestamp_prompt = (
        "Call the timestamp_grammar to save a timestamp for August 7th 2025 at 10AM."
)

response_mssql = client.responses.create(
    model="gpt-5",
    input=timestamp_prompt,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "timestamp_grammar",
            "description": "Saves a timestamp in date + time in 24-hr format.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": timestamp_grammar_definition
            }
        },
    ],
    parallel_tool_calls=False
)

print("--- Timestamp ---")
print(response_mssql.output[1].input)

--- Timestamp ---
2025-08-07 10:00


### 3.5 ベストプラクティス

Larkの文法を完璧にするのは難しい場合があります。シンプルな文法が最も確実に動作する一方で、複雑な文法では、モデルが分布から外れないようにするために、文法定義自体、プロンプト、ツールの説明を繰り返し調整する必要があることが多いです。

- 終端記号を制限する – `/.*\./`ではなく`/[^.\n]{0,10}*\./`を使用する。内容（否定文字クラス）と長さ（`{M,N}`量詞）の両方でマッチを制限する。
- `.`ワイルドカードよりも明示的な文字クラスを優先する。
- グローバルな`%ignore`の代わりに、例えば`SP = " "`を使用して、空白文字を明示的に処理する。
- ツールを説明する：CFGが何を受け入れるかをモデルに正確に伝え、準拠について十分に推論するよう指示する。

**トラブルシューティング**
- APIが文法が複雑すぎるとして拒否する ➜ ルールと終端記号を簡素化し、`%ignore.*`を削除する。
- 予期しないトークン ➜ 終端記号が重複していないことを確認し、貪欲レキサーをチェックする。
- モデルが「分布外」に逸脱する場合（過度に長いまたは反復的な出力を生成することで現れ、構文的には有効だが意味的に間違っている）：
    - 文法を厳しくする。
    - プロンプト（few-shotの例を追加）とツールの説明（文法を説明し、それに準拠するよう推論するようモデルに指示）を繰り返し調整する。
    - より高い推論努力で実験する（例：mediumからhighに上げる）。

**リソース：**  
- Lark Docs – https://lark-parser.readthedocs.io/en/stable/
- Lark IDE – https://www.lark-parser.org/ide/
- LLGuidance Syntax – https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md
- Regex (Rust crate) – https://docs.rs/regex/latest/regex/#syntax

### 3.6 要点

GPT-5のContext-Free Grammar（CFG）サポートにより、モデルの出力を事前定義された構文に厳密に制約し、有効な文字列のみが生成されることを保証できます。これは、プログラミング言語のルールやカスタム形式を強制する際に特に有用で、後処理やエラーを削減します。正確な文法と明確なツール説明を提供することで、モデルを目標とする出力構造内に確実に留めることができます。

## 4. 最小推論

### 4.1 概要

GPT-5は新しい最小推論努力をサポートしています。最小推論努力を使用する場合、モデルは非常に少ない推論トークンまたは推論トークンを全く出力しません。これは、開発者が最初のユーザー可視トークンまでの時間を非常に高速にしたいユースケース向けに設計されています。注意：推論努力が指定されていない場合、デフォルト値は中程度です。

In [13]:
from openai import OpenAI

client = OpenAI()

prompt = "Classify sentiment of the review as positive|neutral|negative. Return one word only." 


response = client.responses.create(
    model="gpt-5",
    input= [{ 'role': 'developer', 'content': prompt }, 
            { 'role': 'user', 'content': 'The food that the restaurant was great! I recommend it to everyone.' }],
    reasoning = {
        "effort": "minimal"
    },
)

# Extract model's text output
output_text = ""
for item in response.output:
    if hasattr(item, "content"):
        for content in item.content:
            if hasattr(content, "text"):
                output_text += content.text

# Token usage details
usage = response.usage

print("--------------------------------")
print("Output:")
print(output_text)


--------------------------------
Output:
positive


### 4.2 要点

最小推論は、レイテンシを最小化し、最初のトークンまでの時間を短縮するために、推論トークンをほとんど使用しないか全く使用せずにGPT-5を実行します。説明が不要な決定論的で軽量なタスク（抽出、フォーマット、短い書き直し、単純な分類）に使用してください。努力レベルを指定しない場合、デフォルトでmediumになります。熟考よりも速度を重視する場合は、明示的にminimalを設定してください。